## [Computational Social Science] Project 5: Natural Language Processing

In this project, you will use natural language processing techniques to explore a dataset containing tweets from members of the 116th United States Congress that met from January 3, 2019 to January 2, 2021. The dataset has also been cleaned to contain information about each legislator. Concretely, you will do the following:

* Preprocess the text of legislators' tweets
* Conduct Exploratory Data Analysis of the text
* Use sentiment analysis to explore differences between legislators' tweets
* Featurize text with manual feature engineering, frequency-based, and vector-based techniques
* Predict legislators' political parties and whether they are a Senator or Representative

You will explore two questions that relate to two central findings in political science and examine how they relate to the text of legislators' tweets. First, political scientists have argued that U.S. politics is currently highly polarized relative to other periods in American history, but also that the polarization is asymmetric. Historically, there were several conservative Democrats (i.e. "blue dog Democrats") and liberal Republicans (i.e. "Rockefeller Republicans"), as measured by popular measurement tools like [DW-NOMINATE](https://en.wikipedia.org/wiki/NOMINATE_(scaling_method)#:~:text=DW\%2DNOMINATE\%20scores\%20have\%20been,in\%20the\%20liberal\%2Dconservative\%20scale.). However, in the last few years, there are few if any examples of any Democrat in Congress being further to the right than any Republican and vice versa. At the same time, scholars have argued that this polarization is mostly a function of the Republican party moving further right than the Democratic party has moved left. **Does this sort of asymmetric polarization show up in how politicians communicate to their constituents through tweets?**

Second, the U.S. Congress is a bicameral legislature, and there has long been debate about partisanship in the Senate versus the House. The House of Representatives is apportioned by population and all members serve two year terms. In the Senate, each state receives two Senators and each Senator serves a term of six years. For a variety of reasons (smaller chamber size, more insulation from the voters, rules and norms like the filibuster, etc.), the Senate has been argued to be the "cooling saucer" of Congress in that it is more bipartisan and moderate than the House. **Does the theory that the Senate is more moderate have support in Senators' tweets?**

**Note**: See the project handout for more details on caveats and the data dictionary.

In [1]:
# pandas and numpy
import pandas as pd
import numpy as numpy

# punctuation, stop words and English language model
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
nlp = en_core_web_sm.load()

# textblob
from textblob import TextBlob

# countvectorizer, tfidfvectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# gensim
import gensim
from gensim import models

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:
import spacy 
import string
import re

In [3]:
# load data 
# ----------
congress_tweets = pd.read_csv("data/116th Congressional Tweets and Demographics.csv")
# fill in this line of code with a sufficient number of tweets, depending on your computational resources
numpy.random.seed(369)
congress_tweets = congress_tweets.sample(1000)
pd.set_option('display.max_colwidth', 1000)

congress_tweets.head()


,tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share
936437,1.187430e+18,RepGwenMoore,2019-10-24T13:57:14-04:00,"Trump's #gagrule is causing real harm. Not only does this policy jeopardize low-income women’s access to cancer screenings and reproductive care, but the lack of funding could result in higher costs for patients. The ripple effects endanger women. https://www.nbcnews.com/news/latino/nearly-900-women-s-health-clinics-have-lost-federal-funding-n1069591",Gwen Moore,Rep,3-Jan-05,4/18/1951,F,WI,4,Democrat,"1,405,284","1,382,536","1,620,985","1,407,966"
101552,1.205240e+18,RepRoKhanna,2019-12-12T16:33:00-05:00,"Wondering how we are going to pay for #MedicareForAll? Questions about how to keep your doctor, or what zero copay and zero premiums will mean for you? Join me tomorrow at 10am EST for a livestream with policy experts to discuss what the future of American health care looks like.",Ro Khanna,Rep,3-Jan-17,9/13/1976,M,CA,17,Democrat,"4,483,814","8,753,792","7,854,285","4,839,958"
290630,1.216890e+18,RodneyDavis,2020-01-13T19:52:44-05:00,"According to a 2018 report from the Equal Employment Opportunity Commission (EEOC), “women, especially older women, but also those at middle age, were subjected to more age discrimination than older men.” I offered an amendment to study this impact. https://www.youtube.com/watch?v=CK2d6qfYfAo&feature=youtu.be&t=4608",Rodney Davis,Rep,3-Jan-13,1/5/1970,M,IL,13,Republican,"2,146,015","3,090,729","3,019,512","2,135,216"
280629,1.141100e+18,RepCheri,2019-06-18T18:00:36-04:00,Had a great conversation about improving rural health care with @UICnursing in the Quad-Cities yesterday. Bringing down the cost of health care will remain a top priority of mine. Learn more in the @Dispatch_Argus: \n \nhttps://qconline.com/news/local/there-s-a-huge-need-nurses-talk-about-work-in/article_40ee6f12-765f-5686-9a75-5abf893f1560.html#1,Cheri Bustos,Rep,3-Jan-13,10/17/1961,F,IL,17,Democrat,"2,146,015","3,090,729","3,019,512","2,135,216"
812427,1.257780e+18,RepPeteOlson,2020-05-05T17:06:55-04:00,Mental health is extremely important in these challenging times. @NAMIGulfCoast is making sure the people of #BrazoriaCounty know no one is alone. Another terrific example of #TX22GoodWillCreators! http://pbs.twimg.com/media/EXSGVH5WkAAOOgo.jpg,Pete Olson,Rep,3-Jan-09,12/9/1962,M,TX,22,Republican,"4,685,047","3,877,868","3,308,124","4,569,843"


In [4]:
text = congress_tweets['text']

## Preprocessing

The first step in working with text data is to preprocess it. Make sure you do the following:

* Remove punctuation and stop words. The `rem_punc_stop()` function we used in lab is provided to you but you should feel free to edit it as necessary for other steps
* Remove tokens that occur frequently in tweets, but may not be helpful for downstream classification. For instance, many tweets contain a flag for retweeting, or share a URL 

As you search online, you might run into solutions that rely on regular expressions. You are free to use these, but you should also be able to preprocess using the techniques we covered in lab. Specifically, we encourage you to use spaCy's token attributes and string methods to do some of this text preprocessing.

In [5]:
# initalize an empty list
tokens_reduced = []

def rem_punc_stop(text):
    stop_words = STOP_WORDS
    
    punc = set(punctuation)
    
    #I am going to put everything in lowercases to be sure I caught stop words with upper letters. The risk might be loosing context, for instance Congress or emphasis, but that might not be important in my later model 
    punc_free = " ".join([ch.lower() for ch in text if ch not in punc])
    
    
    # Remove emojis 
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    punc_free = emoji_pattern.sub(r'', punc_free)
    

    doc = nlp(punc_free)
    
    spacy_words = [token.text for token in doc]
    
    spacy_words = [token.text for token in doc if not token.like_num]
    
    spacy_words = [word for word in spacy_words if not word.startswith('http')]
    
    no_punc = [word for word in spacy_words if word not in stop_words]
    
     # I will clean the data further, to elimitate repetitive useless terms or text modifiers: RT, QT, and \n , wierdly the punctuation function is not working correctly to remove some signs
    no_punc = [word for word in no_punc if word not in ["RT", "QT", "amp", "\n", "\n\n", "\n \n", "#", "." , "-", ",", ";", "?", "!",":", "'","~"]]
    
    return no_punc

In [6]:
# apply function
# ----------

# apply our new function to our text object from above
tokens_reduced = rem_punc_stop(text)

# view the first 20 tokens
tokens_reduced[0:50]

['trump',
 'gagrule',
 'causing',
 'real',
 'harm',
 'policy',
 'jeopardize',
 'low',
 'income',
 'women',
 'access',
 'cancer',
 'screenings',
 'reproductive',
 'care',
 'lack',
 'funding',
 'result',
 'higher',
 'costs',
 'patients',
 'ripple',
 'effects',
 'endanger',
 'women',
 'wondering',
 'going',
 'pay',
 'medicareforall',
 'questions',
 'doctor',
 'copay',
 'premiums',
 'mean',
 'join',
 'tomorrow',
 'est',
 'livestream',
 'policy',
 'experts',
 'discuss',
 'future',
 'american',
 'health',
 'care',
 'looks',
 'like',
 'according',
 'report',
 'equal']

In [7]:
# Entity recognition 
# first, convert tokens_reduced from list to a string
tokens_reduced_str = str(tokens_reduced)

# next, apply function
for entity in nlp(tokens_reduced_str).ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))
    

tomorrow - DATE - Absolute or relative dates or periods
american - NORP - Nationalities or religious or political groups
yesterday - DATE - Absolute or relative dates or periods
irs - ORG - Companies, agencies, institutions, etc.
covid-19 - PERSON - People, including fictional
today - DATE - Absolute or relative dates or periods
today - DATE - Absolute or relative dates or periods
american - NORP - Nationalities or religious or political groups
china - GPE - Countries, cities, states
today - DATE - Absolute or relative dates or periods
today - DATE - Absolute or relative dates or periods
reagan - PERSON - People, including fictional
tonight - TIME - Times smaller than a day
months - DATE - Absolute or relative dates or periods
colorado - GPE - Countries, cities, states
colorado - GPE - Countries, cities, states
russia - GPE - Countries, cities, states
tim - PERSON - People, including fictional
burnett - PERSON - People, including fictional
months - DATE - Absolute or relative dates or 

In [8]:
for word in nlp(tokens_reduced_str):
    print(word.text,  
          word.lemma_)

[ [
' '
trump trump
' '
, ,
' '
gagrule gagrule
' '
, ,
' '
causing cause
' '
, ,
' '
real real
' '
, ,
' '
harm harm
' '
, ,
' '
policy policy
' '
, ,
' '
jeopardize jeopardize
' '
, ,
' '
low low
' '
, ,
' '
income income
' '
, ,
' '
women woman
' '
, ,
' '
access access
' '
, ,
' '
cancer cancer
' '
, ,
' '
screenings screening
' '
, ,
' '
reproductive reproductive
' '
, ,
' '
care care
' '
, ,
' '
lack lack
' '
, ,
' '
funding fund
' '
, ,
' '
result result
' '
, ,
' '
higher high
' '
, ,
' '
costs cost
' '
, ,
' '
patients patient
' '
, ,
' '
ripple ripple
' '
, ,
' '
effects effect
' '
, ,
' '
endanger endanger
' '
, ,
' '
women woman
' '
, ,
' '
wondering wonder
' '
, ,
' '
going go
' '
, ,
' '
pay pay
' '
, ,
' '
medicareforall medicareforall
' '
, ,
' '
questions question
' '
, ,
' '
doctor doctor
' '
, ,
' '
copay copay
' '
, ,
' '
premiums premium
' '
, ,
' '
mean mean
' '
, ,
' '
join join
' '
, ,
' '
tomorrow tomorrow
' '
, ,
' '
est est
' '
, ,
' '
livestream livestream
'

guidance guidance
' '
, ,
' '
blessed bless
' '
, ,
' '
natural natural
' '
, ,
' '
resources resource
' '
, ,
' '
like like
' '
, ,
' '
backyard backyard
' '
, ,
' '
better well
' '
, ,
' '
time time
' '
, ,
' '
explore explore
' '
, ,
' '
rt rt
' '
, ,
' '
@brithume @brithume
' '
, ,
' '
quote quote
' '
, ,
' '
born bear
' '
, ,
' '
alive alive
' '
, ,
' '
abortion abortion
' '
, ,
' '
survivors survivor
' '
, ,
' '
protection protection
' '
, ,
' '
act act
' '
, ,
' '
“ "
' '
, ,
' '
dozen dozen
' '
, ,
' '
states state
' '
, ,
' '
currently currently
' '
, ,
' '
illegal illegal
' '
, ,
' '
doctor doctor
' '
, ,
' '
allow allow
' '
, ,
' '
newborn newborn
' '
, ,
' '
survives survive
' '
, ,
' '
abortion abortion
' '
, ,
' '
die die
' '
, ,
' '
neglect neglect
' '
, ,
' '
democrats democrats
' '
, ,
' '
blocked block
' '
, ,
' '
bill bill
' '
, ,
' '
changed change
' '
, ,
' '
” "
' '
, ,
' '
remember remember
' '
, ,
' '
love love
' '
, ,
' '
honor honor
' '
, ,
' '
action action
'

tax tax
' '
, ,
' '
return return
' '
, ,
' '
& &
' '
, ,
' '
dependents dependent
' '
, ,
' '
recipients recipient
' '
, ,
' '
dependents dependent
' '
, ,
' '
use use
' '
, ,
' '
non non
' '
, ,
' '
filers filer
' '
, ,
' '
tool tool
' '
, ,
' '
today today
' '
, ,
' '
$ $
' '
, ,
' '
eligible eligible
' '
, ,
' '
child child
' '
, ,
' '
added add
' '
, ,
' '
impact impact
' '
, ,
' '
payment payment
' '
, ,
' '
today today
' '
, ,
' '
day day
' '
, ,
' '
register register
' '
, ,
' '
wait wait
' '
, ,
' '
bring bring
' '
, ,
' '
american american
' '
, ,
' '
companies company
' '
, ,
' '
home home
' '
, ,
' '
act act
' '
, ,
' '
reduce reduce
' '
, ,
' '
dependence dependence
' '
, ,
' '
china china
' '
, ,
' '
attract attract
' '
, ,
' '
investment investment
' '
, ,
' '
home home
' '
, ,
' '
common common
' '
, ,
' '
sense sense
' '
, ,
' '
solution solution
' '
, ,
' '
bring bring
' '
, ,
' '
companies company
' '
, ,
' '
increase increase
' '
, ,
' '
national national
' '
, ,
' 

## Exploratory Data Analysis

Use two of the techniques we covered in lab (or other techniques outside of lab!) to explore the text of the tweets. You should construct these visualizations with an eye toward the eventual classification tasks: (1) predicting the legislator's political party based on the text of their tweet, and (2) predicting whether the legislator is a Senator or Representative. As a reminder, in lab we covered word frequencies, word clouds, word/character counts, scattertext, and topic modeling as possible exploration tools. 

### EDA 1

In [ ]:
Lets make a word cloud. 


### EDA 2

In [ ]:
Lets make the topic modelling  

## Sentiment Analysis

Next, let's analyze the sentiments contained within the tweets. You may use TextBlob or another library for these tasks. Do the following:

* Choose two legislators, one who you think will be more liberal and one who you think will be more conservative, and analyze their sentiment and/or subjectivity scores per tweet. For instance, you might do two scatterplots that plot each legislator's sentiment against their subjectivity, or two density plots for their sentiments. Do the scores match what you thought?
* Plot two more visualizations like the ones you chose in the first part, but do them to compare (1) Democrats v. Republicans and (2) Senators v. Representatives 

`TextBlob` has already been imported in the top cell.

In [ ]:
...

## Featurization

Before going to classification, explore different featurization techniques. Create three dataframes or arrays to represent your text features, specifically:

* Features engineered from your previous analysis. For example, word counts, sentiment scores, topic model etc.
* A term frequency-inverse document frequency matrix. 
* An embedding-based featurization (like a document averaged word2vec)

In the next section, you will experiment with each of these featurization techniques to see which one produces the best classifications.

In [ ]:
...

### Engineered Text Features

In [ ]:
# Engineered Features
...

### Bag-of-words or Tf-idf

In [ ]:
# Frequency Based featurization
...

### Word Embedding

In [ ]:
# Load Word2Vec model from Google; OPTIONAL depending on your computational resources (the file is ~1 GB)
# Also note that this file path assumes that the word vectors are underneath 'data'; you may wish to point to the CSS course repo and change the path
# or move the vector file to the project repo 

#model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary = True) 

In [ ]:
# Function to average word embeddings for a document; use examples from lab to apply this function. You can use also other techniques such as PCA and doc2vec instead.
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

In [ ]:
# embedding based featurization
...

## Classification

Either use cross-validation or partition your data with training/validation/test sets for this section. Do the following:

* Choose a supervised learning algorithm such as logistic regression, random forest etc. 
* Train six models. For each of the three dataframes you created in the featurization part, train one model to predict whether the author of the tweet is a Democrat or Republican, and a second model to predict whether the author is a Senator or Representative.
* Report the accuracy and other relevant metrics for each of these six models.
* Choose the featurization technique associated with your best model. Combine those text features with non-text features. Train two more models: (1) A supervised learning algorithm that uses just the non-text features and (2) a supervised learning algorithm that combines text and non-text features. Report accuracy and other relevant metrics. 

If time permits, you are encouraged to use hyperparameter tuning or AutoML techniques like TPOT, but are not explicitly required to do so.

### Train Six Models with Just Text

In [ ]:
# six models ([engineered features, frequency-based, embedding] * [democrat/republican, senator/representative])
...

### Two Combined Models

In [ ]:
# two models ([best text features + non-text features] * [democrat/republican, senator/representative])
...

## Discussion Questions

1. Why do standard preprocessing techniques need to be further customized to a particular corpus?

**YOUR ANSWER HERE** ...

2. Did you find evidence for the idea that Democrats and Republicans have different sentiments in their tweets? What about Senators and Representatives?

**YOUR ANSWER HERE** ...

3. Why is validating your exploratory and unsupervised learning approaches with a supervised learning algorithm valuable?

**YOUR ANSWER HERE** ...

4. Did text only, non-text only, or text and non-text features together perform the best? What is the intuition behind combining text and non-text features in a supervised learning algorithm?

**YOUR ANSWER HERE** ...